In [1]:
import os

In [2]:
os.chdir('../../')
!ls

README.md      data    records	setup_environment.sh
conda_env.txt  models  run.sh	src


In [3]:
import sys
sys.path.insert(0,'./src/')

In [4]:
from src.model import *

In [5]:
from transformers import AutoTokenizer
# Make up some training data
training_data = [
    (
    "测 试 数 据".split(),
    "O O O O".split()
    ), 
    (
    "网 易 公 司".split(),
    "B-ORG I-ORG I-ORG I-ORG".split()
    )
]

tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese", cache_dir = './models/bert/')
b_x = []
b_y = []
tag_encoder = TagEncoder()
model_config = {
    'base_model_name': 'bert-base-chinese',
    'cache_dir': './models/bert/',
    'hidden_size': 768,
    'hidden_dropout_prob': 0.1,
}
model = BertCRFForNER(model_config)
model.eval()
with torch.no_grad():
    for x,y in training_data:
        x = tokenizer.convert_tokens_to_ids(x)
        y = tag_encoder.single_to_ids(y)
        b_x.append(x)
        b_y.append(y)
        x = torch.tensor([x], dtype=torch.long)
        print(model.single_forward(x))

    print('-'*30)
    print(b_x)
    print(b_y)
    print(model.batch_forward(torch.tensor(b_x, dtype=torch.long)))

(tensor(5.3656), [2, 4, 2, 0])
(tensor(5.3371), [6, 7, 4, 3])
------------------------------
[[3844, 6407, 3144, 2945], [5381, 3211, 1062, 1385]]
[[7, 7, 7, 7], [3, 4, 4, 4]]
([tensor(5.3656), tensor(5.3371)], [[2, 4, 2, 0], [6, 7, 4, 3]])


In [6]:
from torch import optim
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)
from tqdm import tqdm

model.train()
# Make sure prepare_sequence from earlier in the LSTM section is loaded
for epoch in tqdm(range(10)):  # again, normally you would NOT do 300 epochs, it is toy data
    b_x = []
    b_y = []
    model.zero_grad()
    # Step 1. Remember that Pytorch accumulates gradients.
    # We need to clear them out before each instance
    
    for sentence, tags in training_data:
        x = tokenizer.convert_tokens_to_ids(sentence)
        y = tag_encoder.single_to_ids(tags)
        b_x.append(x)
        b_y.append(y)
    
    # Step 2. Get our inputs ready for the network, that is,
    # turn them into Tensors of word indices.
    b_x = torch.tensor(b_x, dtype=torch.long)
    atten_mask = torch.ones_like(b_x)
    b_y = torch.tensor(b_y, dtype=torch.long)

    # Step 3. Run our forward pass.
    loss = model.batch_neg_log_likelihood(b_x, atten_mask, b_y)

    # Step 4. Compute the loss, gradients, and update the parameters by
    # calling optimizer.step()
    loss.backward()
    optimizer.step()
    

100%|██████████| 10/10 [00:35<00:00,  3.50s/it]


In [7]:
b_x = []
b_y = []
model.eval()
with torch.no_grad():
    for x,y in training_data:
        x = tokenizer.convert_tokens_to_ids(x)
        y = tag_encoder.single_to_ids(y)
        b_x.append(x)
        b_y.append(y)
        x = torch.tensor([x], dtype=torch.long)
        print(model.single_forward(x))

    print('-'*30)
    print(b_x)
    print(b_y)
    print(model.batch_forward(torch.tensor(b_x, dtype=torch.long)))

(tensor(22.5156), [7, 7, 7, 7])
(tensor(22.5157), [7, 7, 7, 7])
------------------------------
[[3844, 6407, 3144, 2945], [5381, 3211, 1062, 1385]]
[[7, 7, 7, 7], [3, 4, 4, 4]]
([tensor(22.5156), tensor(22.5157)], [[7, 7, 7, 7], [7, 7, 7, 7]])
